<a href="https://colab.research.google.com/github/A01720548/M1-Actividad-Multiagentes/blob/main/M1_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esteban de la Maza A01720548

In [1]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 25.6 MB/s 
     |████████████████████████████████| 60 kB 7.1 MB/s 


In [2]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `Multigrid` que nos deja tener varios agentes en una celda
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

import signal

In [136]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for agent in cell_content:
          # if agent.unique_id[2] == 2:
            grid[x][y] = agent.status


    return grid

class VaccuumAgent(Agent):
    def __init__(self, unique_id, model):
      super().__init__(unique_id, model)
      self.status = 2

    def random_move(self):
      '''
      Este metodo nos entrega de manera aleatoria una lista de posiciones donde se pueda mover 
      de una cuadricula a la vez y desplaza a los objetos de tipo Vaccuum 
      '''
      next_moves = self.model.grid.get_neighborhood(self.pos, True, True)
      next_move = self.random.choice(next_moves)
      # Realizar el movimiento
      self.model.grid.move_agent(self, next_move)

    def step(self):
      '''
      Este método es el que calcula si esta limpio o sucio el piso de la posicion donde se encuentra actualmente
      a traves de su unique ID y una verificacion de que tipo de objeto es. En caso de que se tenga que limpiar, 
      se actualiza el estado del piso 
      '''
      neighbours = self.model.grid.get_neighbors(
          self.pos,
          moore=False,
          include_center=True)
      
      for neighbour in neighbours:
        if (neighbour.unique_id[0] == self.pos[0] and neighbour.unique_id[1] == self.pos[1] and neighbour.unique_id[2] == 1 and neighbour.status == 1):
          neighbour.status = 0
        
    def advance(self):
      '''
      Define el nuevo estado despues de llamar a la funcion para moverse de manera aleatoria
      '''
      self.random_move()


class FloorAgent(Agent):
    '''
    Representa a un agente o una celda con estado sucio (1) o limpio (0)
    '''
    def __init__(self, unique_id, model, isDirty):
      '''
      Crea un agente con estado inicial aleatorio de 0 o 1, también se le asigna un identificador 
      formado por una tupla (x,y,1). 
      '''
      super().__init__(unique_id, model)
      self.status = isDirty
       

class VaccuumAgentModel(Model):
    '''
    Define el modelo donde se encuentran los agentes de aspiradoras y piso. Se crea el MultiGrid y se agregan agentes
    a las casillas. Si es tipo vaccuum su posicion inicial es (1,1) y se agrega un 0 para identificarlo como Vaccuum
    Si es tipo Floor se agrega a cada casilla y se le da un 1 en su Unique ID para identificarlo
    '''
    def __init__(self, width, height, numAgents, percentageDirty):
      self.num_agents = width * height
      self.grid = MultiGrid(width, height, True)
      self.schedule = SimultaneousActivation(self)
      
      # Percentage of Initial Dirty Floors
      gridSize = width*height
      total = int(gridSize * percentageDirty/100)
      # print(total)
      arrayOfOnes = np.zeros((width,height), int)
      x = 0
      while x < total:
        pos = np.random.randint(0,width-1)
        pos2 = np.random.randint(0,height-1)
        if arrayOfOnes[pos][pos2] == 0:
          arrayOfOnes[pos][pos2] = 1
          x+=1
      # print(arrayOfOnes)
      for (content, x, y) in self.grid.coord_iter():
        if numAgents > 0:
          a = VaccuumAgent((x, y, 0), self)
          self.grid.place_agent(a, (1,1)) # Inicializar los Agentes en la celda 1,1
          self.schedule.add(a)
          numAgents-=1
        b = FloorAgent((x,y, 1), self, arrayOfOnes[x][y])
        self.grid.place_agent(b, (x,y))
        self.schedule.add(b)

      
      # Aquí definimos con colector para obtener el grid completo.
      self.datacollector = DataCollector(
          model_reporters={"Grid": get_grid})
    
    def allClean(self):
      cleanCells = 0
      for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for agent in cell_content:
          if agent.unique_id[2] == 1 and agent.status == 0:
            cleanCells +=1
      
      self.percetageClean = cleanCells / (self.grid.width * self.grid.height)
      if self.percetageClean == 1 :
        return 1
      else:
        return self.percetageClean



    def step(self):
      '''
      En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
      '''
      self.datacollector.collect(self)
      self.schedule.step()

In [137]:
# Definimos el tamaño del Grid
GRID_SIZE = 50

# Definimos el número de agentes limpiadores
num_agents = 50

# Definimos el Porcentaje de Celdas Sucias Inicialmente
percentage_dirty = 50

# Max RunTime
start_time = time.time()
maxTime = 1

frameCounter = 0

model = VaccuumAgentModel(GRID_SIZE, GRID_SIZE, num_agents, percentage_dirty)

while(time.time() - start_time) < maxTime and model.allClean() != 1:
  frameCounter+=1
  model.step()



# Tiempo de Ejecucion
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

# Porcentaje de Celas Limpias despues del Termino de la Simulacion
print('Porcentaje de Celas Limpias: ', str(model.allClean() * 100), '%')

# Numero de Movimientos Realizados por los Agentes
print('Numero de Movimientos Realizados: ', frameCounter)


Tiempo de ejecución: 0:00:01.003584
Porcentaje de Celas Limpias:  70.8 %
Numero de Movimientos Realizados:  167


In [138]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [139]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.Blues)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=frameCounter)

In [140]:
anim


# Conclusiones

Probe de 2 agentes a 50 agentes y me di cuenta que el tiempo se afecta de la siguiente manera
